In [8]:
import json
import requests
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient

In [5]:
url = 'https://quotes.toscrape.com/'
quotes_list = []
with open('quotes_data_tmp.json', 'w') as jf:
    while True:
        #print(f'start scraping url {url}')
        r = requests.get(url)
        if r.status_code == 200:
            html = bs(r.content, 'html.parser')
            all_quotes = html.select('div[class=quote]')
            for quote_number, quote in enumerate(all_quotes, 1):
                content = quote.find('span', class_='text').text
                author = quote.select_one('small[class=author]').text
                tags = quote.select_one('meta[class=keywords]')['content'].replace(',',', ')

                quote_dict = {
                    'quote': content,
                    'author': author,
                    'tags': tags      
                }
                quotes_list.append(quote_dict)
                
                json.dump(quote_dict, jf)
                #print(f'\t\tSaved quote #{quote_number} from {len(all_quotes)}')
            try:
                next_page = html.select_one('li[class=next] a')['href']
                url = 'https://quotes.toscrape.com' + next_page
                continue
            except:
                break
        else:
            print(f'Error: {req.status_code}')
            
with open('quotes_data.json', 'w') as jf:
    json.dump(quotes_list, jf)
    
print('Done')

Done


In [9]:
len(quotes_list)

100

1.	Сохранить данные из предыдущего домашнего задания в файл .json или .csv. 
2.	Создать MongoDB, записать данные туда (любое название базы, любое название коллекции). Выполнить команду для демонстрации содержимого коллекции. Прикрепить скриншот.
3.	Создать базу данных sqlite, загрузить туда данные из парсера с предыдущего урока. Загрузить файл .db


In [10]:
client = MongoClient('mongodb://localhost:27017/')

In [11]:
print(client.list_database_names())

['admin', 'cars', 'config', 'local', 'quotes', 'test']


In [13]:
db = client.quotes

In [14]:
print(db.list_collection_names())

['users', 'all_quotes']


In [15]:
db.users.drop()

In [16]:
db.all_quotes.drop()

In [19]:
db.all_quotes.insert_many(quotes_list)

In [20]:
print(db.list_collection_names())

['all_quotes']


In [22]:
db_quotes = db.all_quotes.find()
for quote in db_quotes:
    print(quote)

{'_id': ObjectId('627915bfcc5a80f4f2875472'), 'quote': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”', 'author': 'Albert Einstein', 'tags': 'change, deep-thoughts, thinking, world'}
{'_id': ObjectId('627915bfcc5a80f4f2875473'), 'quote': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”', 'author': 'J.K. Rowling', 'tags': 'abilities, choices'}
{'_id': ObjectId('627915bfcc5a80f4f2875474'), 'quote': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”', 'author': 'Albert Einstein', 'tags': 'inspirational, life, live, miracle, miracles'}
{'_id': ObjectId('627915bfcc5a80f4f2875475'), 'quote': '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”', 'author': 'Jane Austen', 'tags': 'aliteracy, books, classic, humor'}
{'_id': ObjectId('627915bfcc5a80f4f28754

In [26]:
n_quotes = len(list(db.all_quotes.find()))
print("There are {} quotes.".format(n_quotes))

There are 100 quotes.


In [35]:
import sqlite3

In [36]:
connection = sqlite3.connect('sqlite.db')

In [50]:
cursor = connection.cursor()

In [51]:
#method 1

In [44]:
cursor.execute("CREATE TABLE quotes (quotes_list json)")

OperationalError: table all_quotes already exists

In [45]:
cursor.execute('SELECT * FROM all_quotes ')
rows = cursor.fetchall()
print(len(rows))

0


In [ ]:
#method 2

In [46]:
query = '''
    CREATE TABLE quotes(
        "quote" TEXT,
        "author" TEXT,
        "tags" TEXT,
        "_id" TEXT
    )

'''

cursor.execute(query)
connection.commit()

In [49]:
insert_query = '''
    INSERT INTO quotes(
        "quote",
        "author",
        "tags",
        "_id"
    )
    VALUES(
        ?,
        ?,
        ?,
        ?
    )
'''
cursor.executemany(insert_query, quotes_list)
connection.commit()

ProgrammingError: Binding 1 has no name, but you supplied a dictionary (which has only names).